In [1]:
from __future__ import absolute_import
from __future__ import print_function
import tensorflow as tf
import numpy as np  # linear alg
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import pickle
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
#from keras.optimizers import RMSprop
#from tensorflow.keras.optimizers import RMSprop
from keras import backend as K
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
import keras
import matplotlib.pyplot as plt
from keras.models import Model

In [2]:
#epochs = 5
random.seed(25)

In [3]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

In [4]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [5]:
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
      uses y_true as Y and y_pred as the Euclidean distance between dissimilar points
    '''
    print("yes, contrastive")
    margin = 1
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)

In [6]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = input
    
    x = Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape,padding='same')(x)
    x= tf.keras.layers.AveragePooling2D()(x)
    x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
    x= tf.keras.layers.AveragePooling2D()(x)
    x = Conv2D(32, kernel_size=3, activation='relu', padding='same')(x)
    x= tf.keras.layers.AveragePooling2D()(x)
    x = Flatten()(x) 
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    
    x = Lambda(lambda x: K.l2_normalize(x, axis=1))(x) 
    return Model(input, x)

In [7]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

In [8]:
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [9]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    new_var = this_array.copy()
    rat = (this_max - this_min)/(new_var.max() - new_var.min())
    new_var = new_var * rat
    new_var -= new_var.min()
    new_var += this_min
    if set_to_int:
        return new_var.astype('uint8')
    return new_var

In [10]:
#train = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Malignancy/train.csv')
train = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Spiculation/train.csv')
#test = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Malignancy/test.csv')
test = pd.read_csv('/Users/amalalmansour/Desktop/New_Images/Spiculation/test.csv')

In [11]:
train_nodIds = train['noduleID']
test_nodIds = test['noduleID']

In [12]:
train_nodIds = np.array(train_nodIds)
test_nodIds = np.array(test_nodIds)

In [13]:
image_folder = '/Users/amalalmansour/Desktop/images_1332'
train_images = []
test_images = []
train_noduleId = []
test_noduleId = []
print(image_folder)

for dir1 in os.listdir(image_folder):
    print(dir1)
    if not dir1.startswith('.'):
        for file in os.listdir(os.path.join(image_folder, dir1)):
            #print(file)
            noduleID = file.split('.')[0]
            if not file.startswith('.'):
                noduleID = int(noduleID)
                if noduleID in train_nodIds:
                    train_noduleId.append(noduleID)
                    temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
                    temp_image = getNormed(temp_image)
                    #enlarged_img = getSamePadding(temp_image)
                    train_images.append(temp_image)
            
                if noduleID in test_nodIds:
                    test_noduleId.append(noduleID)
                    temp_image = np.loadtxt(os.path.join(image_folder, dir1,file))
                    temp_image = getNormed(temp_image)
                    #enlarged_img = getSamePadding(temp_image)
                    test_images.append(temp_image)

/Users/amalalmansour/Desktop/images_1332
.DS_Store
Malignancy_2
Malignancy_5
Malignancy_4
Malignancy_3
Malignancy_1


In [14]:
tr_df = pd.DataFrame(train_noduleId,columns =['noduleID'])
print(tr_df)

     noduleID
0        2412
1        2374
2         471
3        1907
4        1496
..        ...
577       440
578       657
579      2609
580       904
581       290

[582 rows x 1 columns]


In [15]:
te_df = pd.DataFrame(test_noduleId,columns =['noduleID'])
print(te_df)

     noduleID
0        1252
1        1497
2        2564
3        2558
4        1958
..        ...
140      2421
141      1869
142      1698
143       643
144      2594

[145 rows x 1 columns]


In [16]:
df = pd.read_csv("/Users/amalalmansour/Desktop/New_Images/LIDC_LabelANDRating.csv")
display(df)

,noduleID,InstanceID,Binary_Rating_Mal,Binary_Rating_Spic
0,1,4,1,3
1,5,118,3,0
2,3,134,3,3
3,4,143,3,3
4,6,175,0,0
...,...,...,...,...
1327,2655,40771,3,0
1328,2657,40780,1,3
1329,2656,40794,1,3
1330,2658,40801,3,3


In [17]:
train_slices = pd.merge(tr_df, df)
display(train_slices)

,noduleID,InstanceID,Binary_Rating_Mal,Binary_Rating_Spic
0,2412,36785,0,0
1,2374,36253,3,0
2,471,6853,3,0
3,1907,29039,3,0
4,1496,22427,0,0
...,...,...,...,...
577,440,6517,0,0
578,657,9498,3,0
579,2609,39936,0,0
580,904,13301,0,0


In [18]:
test_slices = pd.merge(te_df, df)
display(test_slices)

,noduleID,InstanceID,Binary_Rating_Mal,Binary_Rating_Spic
0,1252,18823,3,0
1,1497,22434,3,0
2,2564,39328,0,1
3,2558,39203,3,0
4,1958,29602,0,0
...,...,...,...,...
140,2421,36955,0,0
141,1869,28376,0,0
142,1698,25416,0,0
143,643,9251,0,0


In [19]:
#train_ids = train_slices["Binary_Rating_Spic"]
train_labels = train_slices["Binary_Rating_Spic"]
train_ids = train_slices['InstanceID']
#test_ids = test_slices["Binary_Rating_Spic"]
test_labels = test_slices["Binary_Rating_Spic"]
test_ids = test_slices['InstanceID']

In [20]:
train_images = np.array(train_images)
print(len(train_images))
test_images = np.array(test_images)
print(len(test_images))

582
145


In [21]:
# Reshape the data so the pixel vectors turn into arrays, and so the instance id's are also arrays
train_images = train_images.reshape(train_images.shape[0],71,71,1)  
train_ids = np.array(train_ids)
train_labels = np.array(train_labels)
test_images = test_images.reshape(test_images.shape[0], 71, 71,1)
test_ids = np.array(test_ids)

In [22]:
#test data
pickle_in = open("/Users/amalalmansour/Desktop/New_Images/Spiculation/Siamese/pickle/TestPairs1.pickle", "rb")
testPixel = pickle.load(pickle_in)
testPixel = np.array(testPixel)
print(testPixel.shape)
te_pairs = testPixel.reshape(testPixel.shape[0],2,71,71,1)
print(te_pairs.shape)

#test labels
pickle_in = open("/Users/amalalmansour/Desktop/New_Images/Spiculation/Siamese/pickle/TestLabels1.pickle", "rb")
te_y = pickle.load(pickle_in)
te_y = np.array(te_y)
print(te_y.shape)

(12376, 2, 71, 71)
(12376, 2, 71, 71, 1)
(12376,)


In [23]:
'''
train = pd.read_csv(filepath + 'Train_Pixels.csv')
#train = train.drop(['noduleID', 'Dataset','Category'],  axis=1)

test = pd.read_csv(filepath + 'Test_Pixels.csv')
#test = test.drop(['noduleID', 'Dataset','Category'],  axis=1)

train_labels = train["Binary Rating"]

train_ids = train['InstanceID']
train_images = train.drop(['InstanceID', 'Binary Rating'],  axis=1)

test_ids = test['InstanceID']
test_images = test.drop(['InstanceID', 'Binary Rating'], axis=1)
'''

'\ntrain = pd.read_csv(filepath + \'Train_Pixels.csv\')\n#train = train.drop([\'noduleID\', \'Dataset\',\'Category\'],  axis=1)\n\ntest = pd.read_csv(filepath + \'Test_Pixels.csv\')\n#test = test.drop([\'noduleID\', \'Dataset\',\'Category\'],  axis=1)\n\ntrain_labels = train["Binary Rating"]\n\ntrain_ids = train[\'InstanceID\']\ntrain_images = train.drop([\'InstanceID\', \'Binary Rating\'],  axis=1)\n\ntest_ids = test[\'InstanceID\']\ntest_images = test.drop([\'InstanceID\', \'Binary Rating\'], axis=1)\n'

In [24]:
train_pairs = []
train_y = []

In [25]:
train_images.shape[1:]

(71, 71, 1)

In [26]:
input_shape = train_images.shape[1:]
print('input shape ====', input_shape)

input shape ==== (71, 71, 1)


In [27]:
# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

In [28]:
''' 
because we re-use the same instance `base_network`,
the weights of the network
will be shared across the two branches 
'''
processed_a = base_network(input_a)
processed_b = base_network(input_b)

In [29]:
distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

In [30]:
model = Model([input_a, input_b], distance)  #think we are shaping our model 
model.compile(loss=contrastive_loss, optimizer='adam', metrics=[accuracy])

In [31]:
##### Import Trained Model #####
#model = keras.models.load_model("/Users/amalalmansour/Downloads/REU_SCNN-main/LIDC/Amal_Agreement/Spiculation_slices/max_slices/pickle/my_model_original" , compile = True, custom_objects={"contrastive_loss": contrastive_loss})
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 71, 71, 1)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 71, 71, 1)]  0           []                               
                                                                                                  
 model (Functional)             (None, 512)          1067904     ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['model[0][0]',            

In [32]:
filepath ='/Users/amalalmansour/Desktop/New_Images/Spiculation/Siamese/'
print("loading pairs...")
path = filepath + 'pickle/'
training_loss =[]
test_loss = []

loading pairs...


In [33]:
pickle_in1 = open(path + "TrainPairs1.pickle", "rb")
trainPixel1 = pickle.load(pickle_in1)
trainPixel1 = np.asarray(trainPixel1)
tr_pairs1 = trainPixel1.reshape(trainPixel1.shape[0],trainPixel1.shape[1],71,71,1)
#train_pairs.append(tr_pairs1)

pickle_in2 = open(path + "TrainPairs2.pickle", "rb")
trainPixel2 = pickle.load(pickle_in2)
trainPixel2 = np.asarray(trainPixel2)
tr_pairs2 = trainPixel2.reshape(trainPixel2.shape[0],trainPixel2.shape[1],71,71,1)
#train_pairs.append(tr_pairs2)

pickle_in3 = open(path + "TrainPairs3.pickle", "rb")
trainPixel3 = pickle.load(pickle_in3)
trainPixel3 = np.asarray(trainPixel3)
tr_pairs3 = trainPixel3.reshape(trainPixel3.shape[0],trainPixel3.shape[1],71,71,1)
#train_pairs.append(tr_pairs3)

pickle_in4 = open(path + "TrainPairs4.pickle", "rb")
trainPixel4 = pickle.load(pickle_in4)
trainPixel4 = np.asarray(trainPixel4)
tr_pairs4 = trainPixel4.reshape(trainPixel4.shape[0],trainPixel4.shape[1],71,71,1)

pickle_in5 = open(path + "TrainPairs5.pickle", "rb")
trainPixel5 = pickle.load(pickle_in5)
trainPixel5 = np.asarray(trainPixel5)
tr_pairs5 = trainPixel5.reshape(trainPixel5.shape[0],trainPixel5.shape[1],71,71,1)

# train labels:
pickle_lb1 = open(path + "TrainLabels1.pickle", "rb")
tr_y1 = pickle.load(pickle_lb1)
tr_y1 = np.array(tr_y1)
#train_y.append(tr_y1)

# train labels:
pickle_lb2 = open(path + "TrainLabels2.pickle", "rb")
tr_y2 = pickle.load(pickle_lb2)
tr_y2 = np.array(tr_y2)
#train_y.append(tr_y2)

# train labels:
pickle_lb3 = open(path + "TrainLabels3.pickle", "rb")
tr_y3 = pickle.load(pickle_lb3)
tr_y3 = np.array(tr_y3)
#train_y.append(tr_y3)

# train labels:
pickle_lb4 = open(path + "TrainLabels4.pickle", "rb")
tr_y4 = pickle.load(pickle_lb4)
tr_y4 = np.array(tr_y4)
#train_y.append(tr_y4)

# train labels:
pickle_lb5 = open(path + "TrainLabels5.pickle", "rb")
tr_y5 = pickle.load(pickle_lb5)
tr_y5 = np.array(tr_y5)
#train_y.append(tr_y5)

In [34]:
# epoch = 5
for epoch in range(50):
    
    # Load weights:
    isExist = os.path.exists(path + "Saved_Weights") 
    #print(isExist)
    if isExist == True:
        print("Loading weights...")
        model.load_weights(path + "Saved_Weights/weights.ckpt")
    else:
        print("No weights yet...")

    history = model.fit([tr_pairs1[:, 0], tr_pairs1[:, 1]], tr_y1,
      batch_size=128,
      epochs=epoch+1,
      initial_epoch = epoch,
      shuffle=True,
      validation_data=((te_pairs[:, 0], te_pairs[:, 1]), te_y))
    training_loss += history.history['loss']
    test_loss += history.history['val_loss']
    print(f"{epoch}) Training for {epoch+1} epoch is complete.")

    history = model.fit([tr_pairs2[:, 0], tr_pairs2[:, 1]], tr_y2,
      batch_size=128,
      epochs=epoch+1,
      initial_epoch = epoch,
      shuffle=True,
      validation_data=((te_pairs[:, 0], te_pairs[:, 1]), te_y))
    training_loss += history.history['loss']
    test_loss += history.history['val_loss']
    print(f"{epoch}) Training for {epoch+1} epoch is complete.")

    history = model.fit([tr_pairs3[:, 0], tr_pairs3[:, 1]], tr_y3,
      batch_size=128,
      epochs=epoch+1,
      initial_epoch = epoch,
      shuffle=True,
      validation_data=((te_pairs[:, 0], te_pairs[:, 1]), te_y))
    training_loss += history.history['loss']
    test_loss += history.history['val_loss']
    print(f"{epoch}) Training for {epoch+1} epoch is complete.")
    
    history = model.fit([tr_pairs4[:, 0], tr_pairs4[:, 1]], tr_y4,
      batch_size=128,
      epochs=epoch+1,
      initial_epoch = epoch,
      shuffle=True,
      validation_data=((te_pairs[:, 0], te_pairs[:, 1]), te_y))
    training_loss += history.history['loss']
    test_loss += history.history['val_loss']
    print(f"{epoch}) Training for {epoch+1} epoch is complete.")
    
    history = model.fit([tr_pairs5[:, 0], tr_pairs5[:, 1]], tr_y5,
      batch_size=128,
      epochs=epoch+1,
      initial_epoch = epoch,
      shuffle=True,
      validation_data=((te_pairs[:, 0], te_pairs[:, 1]), te_y))
    training_loss += history.history['loss']
    test_loss += history.history['val_loss']
    print(f"{epoch}) Training for {epoch+1} epoch is complete.")
        
    #base_network.save(path + "my_model")
    #model.save_weights(path + "Saved_Weights/weights.ckpt")
    print("Saving weights...")

No weights yet...
yes, contrastive
yes, contrastive
129/313 [===========>..................] - ETA: 7s - loss: 0.2246 - accuracy: 0.6299

KeyboardInterrupt: 

In [ ]:
len(test_loss)

In [ ]:
model = keras.models.load_model("/Users/amalalmansour/Desktop/New_Images/Spiculation/Siamese/pickle/my_model" , compile = True)

In [ ]:
history_dict = history.history
history_dict

In [ ]:
tr_epoch_loss = []
i = 0
for _ in range(50):
    loss = 0.
    for _ in range(5):
        loss += training_loss[i]
        i+=1
    tr_epoch_loss.append(float(loss)/5)
len(tr_epoch_loss)

In [ ]:
t_epoch_loss = []
i = 0
for _ in range(50):
    loss = 0.
    for _ in range(5):
        loss += test_loss[i]
        i+=1
    t_epoch_loss.append(float(loss)/5)
len(t_epoch_loss)

In [ ]:
epoch_count = range(1,len(tr_epoch_loss)+1)
plt.plot(epoch_count, tr_epoch_loss)
plt.plot(epoch_count, t_epoch_loss)
plt.legend(['Training Loss','Testing Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
y_pred_te_before = model.predict((te_pairs[:, 0], te_pairs[:, 1]))
y_pred_te = (y_pred_te_before - y_pred_te_before.min()) / (y_pred_te_before.max() - y_pred_te_before.min())

In [ ]:
print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))